# 

## nazwa plikow po blascie
query_db_bl

In [1]:
import os
import pandas as pd
from bioinfokit.analys import Fasta
from Bio import SeqIO
# helpers function

def get_fasta(filename):
    n = 0
    sequence = ""
    te_name = ""
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('>'):
                n = n + 1
                if not n == 1:
                    first_n = False
                    yield te_name, sequence
                te_name = line[1:].strip()
                sequence = ""
            else:
                if te_name:
                    sequence = sequence + line.strip()
    yield te_name, sequence

def get_len_fasta(filename):
    counter = {}
    with open(filename) as f:
        te_name = ""
        for line in f.readlines():
            if line.startswith('>'):
                te_name = line[1:].strip()
                counter[te_name] = 0
            else:
                if te_name:
                    counter[te_name] = counter[te_name] + len(line.strip()) - line.count("~")
    return counter

def get_from_te(fasta, te_name):
    te_len = get_len_fasta(fasta)[te_name]
    return int(te_len - te_len * dif_percent / 100)

def get_to_te(fasta, te_name):
    te_len = get_len_fasta(fasta)[te_name]
    return int(te_len + te_len * dif_percent / 100)

def read_fasta(filename):
    return pd.read_csv(filename, sep="\t", header=None).rename(headers_dict, axis=1)

def save_fasta(df, filename):
    df.to_csv(filename, sep='\t', index=False, encoding='utf-8', header=False)








In [184]:
ont = ["K10B", "K10f", "LK5_1_3"]
ont = ["ccc"]
ont = ["K10B", "K10f"]
ont = [
    "K10B",
    "K10f",
    "LK5_1_3"
]
genome_path = "genome/DCARv3.4.fa"
genome_path = "genome/genomic2.fna"
te_path = "te/Alex1_3.fas"
dif_percent = 30

te_name = 'Alex1'
te_len = get_len_fasta(te_path)[te_name]
from_te = get_from_te(te_path, te_name)
to_te = get_to_te(te_path, te_name)



"""
mask_genome(te_name, te_path, genome_path, True)
correct_first_part(False, True, True)
oxi = correct_second_part()
save_report(oxi)
#generate_second_report("report_Alex1__K10B_K10f", "doro")

"""


'\nmask_genome(te_name, te_path, genome_path, True)\ncorrect_first_part(False, True, True)\noxi = correct_second_part()\nsave_report(oxi)\n#generate_second_report("report_Alex1__K10B_K10f", "doro")\n\n'

In [7]:
headers = ['qseqid', 'sseqid',  'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'expect value', 'bitscore']
headers_dict = {i[0]:i[1] for i in enumerate(headers)}

In [142]:
import random
def rand_split(sequence, splits):
    for splitLen in splits:
        if splitLen > len(sequence):
            break
        yield sequence[:splitLen]
        sequence = sequence[splitLen::]

def rand_gen(int_list):
    while True:
        yield int(random.choice(int_list))

def cut_genome(genome_path, ont_path, coverage, filename):
    possibility_len = list(get_len_fasta(ont_path).values())
    n = 0
    for i in range(coverage):
        print(i)
        for name, genome_sequence in get_fasta(genome_path):
            print(name)
            with open(filename, 'a') as file:
                for seq in rand_split(genome_sequence, rand_gen(possibility_len)):
                    n = n + 1
                    file.write(f">{name}_{i}_{n}\n{seq}\n")
cut_genome(genome_path, "ont/K10f.fasta", 20, 'cutted_genome.fasta')

0
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
1
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
2
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
3
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
4
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
5
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
6
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
7
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
8
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
9
chromoso

In [185]:
def mask_genome(te_name, te_path, genome_path, make_db=True):
    os.system(f"mkdir -p masked_genome")
    if make_db:
        os.system(f"makeblastdb -in {genome_path} -dbtype nucl -out genome/genome")
    os.system(f"blastn -num_threads 20 -outfmt 6 -query {te_path} -db genome/genome -out masked_genome/{te_name}_genome.bl -perc_identity 0.9")
    te_genome_bl = read_fasta(f"masked_genome/{te_name}_genome.bl")
    te_genome_bl = te_genome_bl[te_genome_bl.qseqid == te_name]
    te_genome_bl["start"] = te_genome_bl['sstart'].where(te_genome_bl['sstart'] <= te_genome_bl["send"], other=te_genome_bl['send'])
    te_genome_bl["stop"] = te_genome_bl['sstart'].where(te_genome_bl['sstart'] > te_genome_bl["send"], other=te_genome_bl['send'])
    # todo
    save_fasta(te_genome_bl[["sseqid", "start", "stop"]], f"masked_genome/to_mask_genome_{te_name}.txt")
    plus_te_genome_bl = te_genome_bl[te_genome_bl.sstart < te_genome_bl.send]
    minus_te_genome_bl = te_genome_bl[te_genome_bl.sstart > te_genome_bl.send]

    merge_headers = ["sseqid", "sstart", "send"]
    save_fasta(plus_te_genome_bl[merge_headers].sort_values(by=merge_headers), f"masked_genome/plus_to_mask_genome_{te_name}.bed")
    merge_headers = ["sseqid", "send", "sstart"]
    save_fasta(minus_te_genome_bl[merge_headers].sort_values(by=merge_headers), f"masked_genome/minus_to_mask_genome_{te_name}.bed")

    os.system(f"bedtools merge -i masked_genome/plus_to_mask_genome_{te_name}.bed -d 200 > masked_genome/plus_to_mask_genome_{te_name}.bed.merged")
    os.system(f"bedtools merge -i masked_genome/minus_to_mask_genome_{te_name}.bed -d 200 > masked_genome/minus_to_mask_genome_{te_name}.bed.merged")
    minus_merged =  pd.read_csv(f"masked_genome/minus_to_mask_genome_{te_name}.bed.merged", sep="\t", header=None)
    plus_merged =  pd.read_csv(f"masked_genome/plus_to_mask_genome_{te_name}.bed.merged", sep="\t", header=None)
    merged_genome = pd.concat(
        [
            minus_merged[
                (minus_merged[2] - minus_merged[1] > from_te)
                & (minus_merged[2] - minus_merged[1] < to_te)
            ],
            plus_merged[
                (plus_merged[2] - plus_merged[1] > from_te)
                & (plus_merged[2] - plus_merged[1] < to_te)
            ],
        ]
    )
    merged_genome["length"] = merged_genome[2] - merged_genome[1]
    save_fasta(merged_genome, f"masked_genome/to_mask_genome_{te_name}.bed.merged")
    os.system(f"bedtools maskfasta -fi {genome_path} -bed masked_genome/to_mask_genome_{te_name}.bed.merged -fo genome/{te_name}_masked_genome")
    return merged_genome
masked_genome = mask_genome(te_name, te_path, genome_path, True)



Building a new DB, current time: 05/14/2024 02:27:49
New DB name:   /home/kiki/data2/genome/genome
New DB title:  genome/genomic2.fna
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kiki/data2/genome/genome
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 9 sequences in 6.73869 seconds.




In [170]:
masked_genome

,0,1,2,length
1,chromosome2,30539632,30544357,4725
4,chromosome3,27717880,27722606,4726
6,chromosome5,8967214,8971939,4725
9,chromosome9,9220466,9226148,5682
10,chromosome9,28529339,28534568,5229
0,chromosome1,18795334,18799058,3724
2,chromosome3,41159456,41164182,4726
5,chromosome4,35827649,35832374,4725


In [64]:
# first part
def  create_subsequent_fasta(fasta, bed_df, file_base):
    # create subsequents with left and right flanks
    # site p or m
    num = 4000

    print(fasta)
    records = SeqIO.parse(fasta, 'fasta')
    record_dict = SeqIO.to_dict(records)
    for bed_rec in  bed_df.itertuples():
        seq_id = bed_rec.sseqid
        record = record_dict[seq_id]
        
        te_start = bed_rec.sstart
        te_end = bed_rec.send

        left_start = te_start - num if te_start - num > 0 else 0
        left_end = te_start

        right_start = te_end
        right_end = te_end + num if te_end + num < len(record) else len(record)

        left_subseq_record = record[left_start:left_end]  # For the full record (with header)
        right_subseq_record = record[right_start:right_end]

        with open(f"first/left_{file_base}", 'a') as file:
                file.write(f">{record.id}\n{left_subseq_record.seq}\n")
        with open(f"first/right_{file_base}", 'a') as file:
                file.write(f">{record.id}\n{right_subseq_record.seq}\n")

def correct_first_part(fq=True, db=True, do_blast=True):
    os.system("mkdir -p blast")
    os.system("mkdir -p first")
    if fq:
        for file in ont:
            os.system(f"seqtk seq -a ont/{file}.fastq.gz > ont/{file}.fasta")
    if db:
        print('okno')
        for file in ont:
            print(f"makeblastdb -in ont/{file}.fasta -dbtype nucl -out ont/{file}")
            os.system(f"makeblastdb -in ont/{file}.fasta -dbtype nucl -out ont/{file}")
    if do_blast:
        print('okno2')
        for file in ont:
            print(
                f"blastn -num_threads 20 -outfmt 6 -query {te_path} -db ont/{file} " \
                f"-out blast/TE_{file}_{te_name}.bl -dust no -perc_identity 0.9"
            )
            os.system(
                f"blastn -num_threads 20 -outfmt 6 -query {te_path} -db ont/{file} " \
                f"-out blast/TE_{file}_{te_name}.bl -dust no -perc_identity 0.9"
            )

    for o in ont:
        te_ont_bl = read_fasta(f"blast/TE_{o}_{te_name}.bl")
        te_ont_bl = te_ont_bl[te_ont_bl.qseqid.str.contains(te_name)]
        plus_te_ont_bl = te_ont_bl[te_ont_bl.sstart < te_ont_bl.send]
        minus_te_ont_bl = te_ont_bl[te_ont_bl.sstart > te_ont_bl.send]
    
    
        plus_te_ont_bed = plus_te_ont_bl[['sseqid', 'sstart', 'send', 'qseqid']].sort_values(by=['sseqid', 'sstart', 'send'])
        minus_te_ont_bed = minus_te_ont_bl[['sseqid', 'send', 'sstart', 'qseqid']].sort_values(by=['sseqid', 'send', 'sstart'])
    
        save_fasta(minus_te_ont_bed, f'first/m_raw_first_blast_{o}_{te_name}.bed')
        save_fasta(plus_te_ont_bed, f'first/p_raw_first_blast_{o}_{te_name}.bed')

        os.system(f"bedtools merge -i first/p_raw_first_blast_{o}_{te_name}.bed -d 1500 -c 4 -o collapse  > first/p_first_blast_{o}_{te_name}.bed.merged")
        os.system(f"bedtools merge -i first/m_raw_first_blast_{o}_{te_name}.bed -d 1500 -c 4 -o collapse  > first/m_first_blast_{o}_{te_name}.bed.merged")

        print("first part")
    
        plus_merged = pd.read_csv(
            f"first/p_first_blast_{o}_{te_name}.bed.merged", sep="\t", header=None).rename(
            {0:'sseqid', 1:'sstart', 2:'send', 3:'qseqid'}, axis=1
        )
        minus_merged = pd.read_csv(
            f"first/m_first_blast_{o}_{te_name}.bed.merged", sep="\t", header=None).rename(
            {0:'sseqid', 1:'sstart', 2:'send', 3:'qseqid'}, axis=1
        )
    
        plus_filtered_bed = plus_merged[(plus_merged.send - plus_merged.sstart) > from_te]
        minus_filtered_bed = minus_merged[(minus_merged.send - minus_merged.sstart) > from_te]

        # todo in this place

        save_fasta(pd.DataFrame({'id': plus_filtered_bed['sseqid'].unique()}), f'first/p_seq_list_{o}_{te_name}.txt')
        save_fasta(pd.DataFrame({'id': minus_filtered_bed['sseqid'].unique()}), f'first/m_seq_list_{o}_{te_name}.txt')
        #df.to_csv(filename, sep='\t', index=False, encoding='utf-8', header=False)
        print("second part")
    
        #long
        # it's filtered only our fasta
        os.system(f"seqtk subseq ont/{o}.fasta first/p_seq_list_{o}_{te_name}.txt > first/p_normal_{o}_{te_name}.fasta")
        os.system(f"seqtk subseq ont/{o}.fasta first/m_seq_list_{o}_{te_name}.txt > first/m_normal_{o}_{te_name}.fasta")
        print("1 part")

        create_subsequent_fasta(f"first/p_normal_{o}_{te_name}.fasta", plus_filtered_bed,  f"p_{o}_{te_name}.fasta")
        create_subsequent_fasta(f"first/m_normal_{o}_{te_name}.fasta", minus_filtered_bed,  f"m_{o}_{te_name}.fasta")

# Second part of analysis

In [3]:


def mid(records, row, ont_file):
    ont_id = row[0]
    for key, value in records.items():
        if key[1] - row[2] < 1000 and key[1] - row[2] > -1000:
            records[key][0] = records[key][0] + 1
            records[key][1] =   f"{records[key][1]},{ont_file}:{ont_id}"
            return records
    records[row[1], row[2]] =  [1, f"{ont_file}:{ont_id}"]
    return records

def merge(report, records, ont_file):
    #{chromosome: [start or end, number, ont_ids]
    for record in records:
        report = mid(report, record, ont_file)
    return report

    
def merge_results(parts, ont_file, start, end):
    start_te = []
    end_te = []
    for part in parts:
        start_te.extend(part[0])
        end_te.extend(part[1])
    start_results = merge(start, start_te, ont_file)
    end_results = merge(end, end_te, ont_file)
    return start_results, end_results




In [166]:
#second part
def blast_gen(query, db, out):
    # i need only 1 target
    os.system(f"blastn -num_threads 20 -outfmt 6 -query {query} -db {db} -out {out} -perc_identity 0.9 -max_target_seqs 1")
    return pd.read_csv(out, sep="\t", header=None).rename(headers_dict, axis=1)

def _get_ends(df, field_name):
    records = []
    unique_names = set(df.qseqid.unique())
    for name in unique_names:
        df_record = df[df.qseqid == name]
        if len(df_record):
            record = df_record.iloc[0]
            records.append(
                (
                    record['qseqid'],
                    record['sseqid'],
                    record[field_name],
                )
            )
    return records
        
def get_results(left_blast, right_blast):
    te_start = []
    te_end = []

    plus_left = left_blast[left_blast.sstart <= left_blast.send]
    plus_left["strain"] = "+"
    plus_right =  right_blast[right_blast.sstart <= right_blast.send]
    plus_right["strain"] = "+"
    
    minus_left = left_blast[left_blast.sstart > left_blast.send]
    minus_left["strain"] = "-"
    minus_right = right_blast[right_blast.sstart > right_blast.send]
    minus_right["strain"] = "-"


    filtered_plus_left =  plus_left[plus_left.length > 600]
    filtered_minus_right =  minus_right[minus_right.length > 600]

    filtered_plus_right =   plus_right[plus_right.length > 600]
    filtered_minus_left = minus_left[minus_left.length > 600]

    # start te
    te_start.extend(_get_ends(filtered_plus_left, "send"))
    te_start.extend(_get_ends(filtered_minus_right, "send"))
    
    # end te
    te_end.extend(_get_ends(filtered_plus_right, "sstart"))
    te_end.extend(_get_ends(filtered_minus_left, "sstart"))

    return (te_start, te_end)


def correct_second_part(blast_db=True):
    report = {}
    os.system("mkdir -p final")
    os.system("mkdir -p second")
    pd.options.mode.copy_on_write = True
    if blast_db:
        # todo
        os.system(f"makeblastdb -in {genome_path}  -dbtype nucl -out genome/masked_genome")
    for o in ont:
        l_m = blast_gen(
            f"first/left_m_{o}_{te_name}.fasta",
            f"genome/masked_genome",
            f"second/left_m_{o}_{te_name}_genome.bl",
        ).sort_values(['qseqid', 'bitscore'], ascending=False).groupby(['qseqid'], as_index=False).first()
        r_m = blast_gen(
            f"first/right_m_{o}_{te_name}.fasta",
            f"genome/masked_genome",
            f"second/right_m_{o}_{te_name}_genome.bl",
        ).sort_values(['qseqid', 'bitscore'], ascending=False).groupby(['qseqid'], as_index=False).first()
        l_p = blast_gen(
            f"first/left_p_{o}_{te_name}.fasta",
            f"genome/masked_genome",
            f"second/left_p_{o}_{te_name}_genome.bl",
        ).sort_values(['qseqid', 'bitscore'], ascending=False).groupby(['qseqid'], as_index=False).first()
        r_p = blast_gen(
            f"first/right_p_{o}_{te_name}.fasta",
            f"genome/masked_genome",
            f"second/right_p_{o}_{te_name}_genome.bl",
        ).sort_values(['qseqid', 'bitscore'], ascending=False).groupby(['qseqid'], as_index=False).first()

        m_report = get_results(l_m, r_m)
        p_report = get_results(l_p, r_p)

        # todo
        if report:
            report = merge_results([m_report, p_report], o, report[0], report[1])
        else:
            report = merge_results([m_report, p_report], o, {}, {})
    return report

In [13]:
import copy
def get_files():
    files = ""
    for o in ont:
        files = files + f"_{o}"
    return files


def save_report(data):
    start_c = copy.deepcopy(data[0])
    end_c = copy.deepcopy(data[1])
    report = {}
    def get_string(start_te, end_te):
        return end_te - start_te
        return 'reference' if end_te - start_te < 200 else end_te - start_te
    
    for end_key, end_value in data[1].items():
        #del end_c[end_key]
        for start_key, start_value in data[0].items():
            if end_key[0] == start_key[0]:
                if end_key[1] - start_key[1] < 200 and end_key[1] - start_key[1] > -20 :
                    #reference
                    if end_c.get(end_key):
                        del end_c[end_key]
                    if start_c.get(start_key):
                        del start_c[start_key]
                    report[start_key[0], start_key[1], end_key[1]] = start_value + end_value
                if end_key[1] - start_key[1] > from_te and end_key[1] - start_key[1] < to_te:
                    #non reference
                    if end_c.get(end_key):
                        del end_c[end_key]
                    if start_c.get(start_key):
                        del start_c[start_key]
                    report[start_key[0], start_key[1], end_key[1]] = start_value + end_value

    with open(f"report_{te_name}_{get_files()}", "w") as file:
        for key, value in report.items():
                file.write(f"{key[0]}\t{key[1]}\t{key[2]}\t{get_string(key[1], key[2])}\t{value[0]}\t{value[1]}\t{value[2]}\t{value[3]}\n")
        for key, value in start_c.items():
                file.write(f"{key[0]}\t{key[1]}\tNA\tUNCOMPLETE\t{value[0]}\t{value[1]}\tNA\tNA\n")
        for key, value in end_c.items():
                file.write(f"{key[0]}\tNA\t{key[1]}\tNA\tUNCOMPLETE\tNA\t{value[0]}\t{value[1]}\n")

In [214]:
def get_num_and_rec(line, reference):
    start_flag = False
    stop_flag = False
    start_set = set()
    stop_set = set()
    result = {}
    data = line.split('\t')
    result["chromosom"] = data[0]
    result["start"] = data[1]
    result["stop"] = data[2]
    result["length"] = data[3]
    result["pkt"] = 4

    is_reference = reference[
        reference[0] == result["chromosom"]
    ]
    #print(len(is_reference))

    try:
        st = int(data[1])
    except:
        st = 100000000
    try:
        sp = int(data[2])
    except:
        sp = 100000000
    ref = reference[
        (reference[0] == result["chromosom"])
        & (abs(reference[1] - st) < 1000)
        & (abs(reference[2] - sp) < 1000)
    ]
    if len(ref):
        result["is_reference"] = "ref"
    elif abs(st - sp) < 1000:
        result["is_reference"] = "non_ref"
    else:
        result["is_reference"] = "unnown"
    
    for o in ont:
        result[f"num_start_{o}"] = 0
        result[f"rec_start_{o}"] = ""
        result[f"num_stop_{o}"] = 0
        result[f"rec_stop_{o}"] = ""
    try:
        start = data[5]
        stop = data[7]
    except:
        return result
    records_start = start.split(',')
    records_stop = stop.split(',')
    try:
        for record in records_start:
            start_flag = True
            r = record.split(":")
            num_res = result.get(f"num_start_{r[0]}")
            rec_res = result.get(f"rec_start_{r[0]}")
            result[f"num_start_{r[0]}"] = num_res + 1
            result[f"rec_start_{r[0]}"] = rec_res + r[1] if not rec_res else rec_res + ',' + r[1]
            start_set.add(r[1])
    except:
        pass
    try:
        for record in records_stop:
            stop_flag = True
            r = record.split(":")
            num_res = result.get(f"num_stop_{r[0]}")
            rec_res = result.get(f"rec_stop_{r[0]}")
            result[f"num_stop_{r[0]}"] = num_res + 1
            result[f"rec_stop_{r[0]}"] = rec_res + r[1] if not rec_res else rec_res + ',' + r[1]
            stop_set.add(r[1])
    except:
        pass

    if start_flag and stop_flag:
        result["pkt"] = 3
    for o in ont:
        if  result[f"num_start_{o}"] and result[f"num_stop_{o}"]:
            result["pkt"] = 2
    if start_set.intersection(stop_set):
        result["pkt"] = 1
    return result

def generate_second_report(filename_in, filename_out, reference):
    num_rec_header =  "chromosom\tstart\tstop\tlength\tpkt"
    num_rec_header =  "pkt\tchromosom\tstart\tstop\tlength\tis_ref"
    num_rec =  "{chromosom}\t{start}\t{stop}\t{length}\t{pkt}"
    num_rec =  "{pkt}\t{chromosom}\t{start}\t{stop}\t{length}\t{is_reference}"
    for o in ont:
        num_rec_header = num_rec_header +  f"\tnum_start_{o}\trec_start_{o}\tnum_stop_{o}\trec_stop_{o}"
        num_rec = num_rec +  f"\t{{num_start_{o}}}\t{{rec_start_{o}}}\t{{num_stop_{o}}}\t{{rec_stop_{o}}}"
    with open(filename_out, "w") as f_out:
            f_out.write(f"{num_rec_header}\n")
    with open(filename_in) as f_in:
        with open(filename_out, "a") as f_out:
            for line in f_in.readlines():
                f_out.write(
                    f"{num_rec.format(**get_num_and_rec(line, reference))}\n"
                )
generate_second_report("report_Alex1__K10B_K10f_LK5_1_3_old_genome", "old_report", masked_genome)

In [162]:
#oxi

In [173]:
masked_genome

,0,1,2,length
1,chromosome2,30539632,30544357,4725
4,chromosome3,27717880,27722606,4726
6,chromosome5,8967214,8971939,4725
9,chromosome9,9220466,9226148,5682
10,chromosome9,28529339,28534568,5229
0,chromosome1,18795334,18799058,3724
2,chromosome3,41159456,41164182,4726
5,chromosome4,35827649,35832374,4725
